In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import json
import nltk
import pymorphy2
import string
import pickle
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Тимур\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
with open('../profiles/profiles.json', 'r', encoding='utf-8') as file:
    profiles = json.load(file)
profiles[0]

{'city': 'Москва',
 'age': 24,
 'gender': 'Мужчина',
 'preferences': 'Женщины',
 'name': 'Алексей',
 'description': 'Веселый и общительный, люблю путешествовать и открывать новые места. Занимаюсь фотографией, увлекаюсь историей. Ищу девушку с похожими интересами.'}

In [ ]:
descriptions: list[str] = [profile['description'] for profile in profiles]
descriptions[0]

'Веселый и общительный, люблю путешествовать и открывать новые места. Занимаюсь фотографией, увлекаюсь историей. Ищу девушку с похожими интересами.'

In [15]:
morph = pymorphy2.MorphAnalyzer()
rus_stopwords = set(stopwords.words("russian") + ['это', 'который', 'также', 'будут'])


def clean_text(text: str, stopwords_set: set, morph=morph):
    chars = string.punctuation + '«»-—'
    table = str.maketrans("", "", chars)
    new_text = text.replace('\n', ' ').lower()
    new_text = new_text.translate(table)
    cleaned_text = []
    for word in new_text.split():
        lemma = morph.normal_forms(word)[0]
        if not stopwords_set.intersection(set([lemma])):
            cleaned_text.append(lemma)
    return ' '.join(cleaned_text)

In [16]:
descriptions = [clean_text(description, rus_stopwords) for description in descriptions]
descriptions[0]

'весёлый общительный любить путешествовать открывать новый место заниматься фотография увлекаться история искать девушка похожий интерес'

In [18]:
tagged_data = [TaggedDocument(words=word_tokenize(doc.lower(), language='russian'),tags=[str(i)]) for i, doc in enumerate(descriptions)]
tagged_data[0]

TaggedDocument(words=['весёлый', 'общительный', 'любить', 'путешествовать', 'открывать', 'новый', 'место', 'заниматься', 'фотография', 'увлекаться', 'история', 'искать', 'девушка', 'похожий', 'интерес'], tags=['0'])

In [19]:
model = Doc2Vec(vector_size=50, min_count=2, epochs=50)
model.build_vocab(tagged_data)

In [20]:
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.dv.get_normed_vectors()[0]

In [25]:
vectors_profiles = {}
for index, vector in enumerate(model.dv.get_normed_vectors()):
    vectors_profiles[str(vector)] = index

In [23]:
model.save('./model.bin')

In [ ]:
model:Doc2Vec = Doc2Vec.load('./model.bin')

In [28]:
vectors_profiles = {}
for index, vector in enumerate(model.dv.get_normed_vectors()):
    vectors_profiles[pickle.dumps(vector)] = index

In [29]:
vectors_profiles

{b'\x80\x04\x95P\x01\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K2\x85\x94h\x03\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C\xc8\xd3I1\xbc\x95?c\xbe\xdd\x1c"\xbe\x055\xbf\xbdM\xba.\xbc\x1d\x92\x13\xbe\xcb\x88\xf2=\x97\x1aP>(\xbc\x8f\xbe\xaeM\xc8\xbdkF\x0e>\xd0y3\xbe\xd1\x84\xf9=\x10\x85\xaa=;d\x15\xbc\xe0Su=\x05D7>\x91\xa2\xde=\xf6z\xba\xbeC\x82\xa2\xbd\x99b\x17=\x0egW=J<\x9a>\x16V;\xbb\xe4.j>~\xca\x10\xbc\xcb\xbb\x04\xbe\x86 \x05=F^]\xbe\xa7\xf9\x01\xbeC\x01\x8f=5\x82X>\xa4\xa6\x00=\xfc\x10\x84<\xc0\x95\x1a\xbe\x88\xeb\x17>y\xab^=\x1f\xf8\xef\xbcE\xb2\x14\xbd*\xfe\x93=\xed\xd3T>\x10\xbd\xe8;nt\xb4\xbd9\x8d\x01\xbeA\xc6A>\xee\xe7\xec=\x7f#\x17\xbeB\x08\xd1\xbd\xf6\xfe\xdb=tp%\xbc\x94t\x94b.': 0,
 b'\x80\x04\x95P\x01\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94

In [25]:
model.dv.get_normed_vectors()[0]

array([-0.01082082, -0.22192223, -0.15831323, -0.09336285, -0.01066453,
       -0.14411207,  0.11842497,  0.20322643, -0.2807324 , -0.09780441,
        0.1389405 , -0.17526937,  0.12183536,  0.08326161, -0.00911814,
        0.05989444,  0.17897041,  0.10870851, -0.36421937, -0.07935002,
        0.03695926,  0.05258851,  0.30124122, -0.00285852,  0.2286945 ,
       -0.00883734, -0.12962262,  0.03250172, -0.21617994, -0.12692891,
        0.06982663,  0.2114342 ,  0.03140892,  0.01612138, -0.15096188,
        0.14835942,  0.05436275, -0.02929312, -0.03630282,  0.07226212,
        0.20783968,  0.00710262, -0.0881127 , -0.12651528,  0.18923284,
        0.11567675, -0.14759634, -0.10206653,  0.10741989, -0.01009761],
      dtype=float32)

In [ ]:
with open('./vectors_profiles.bin', 'bw') as file:
    pickle.dump(vectors_profiles, file)

In [31]:
profiles_vectors = {}
for index, vector in enumerate(model.dv.get_normed_vectors()):
    profiles_vectors[index] = pickle.dumps(vector)

In [32]:
profiles_vectors[0]

b'\x80\x04\x95P\x01\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K2\x85\x94h\x03\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C\xc8\xd3I1\xbc\x95?c\xbe\xdd\x1c"\xbe\x055\xbf\xbdM\xba.\xbc\x1d\x92\x13\xbe\xcb\x88\xf2=\x97\x1aP>(\xbc\x8f\xbe\xaeM\xc8\xbdkF\x0e>\xd0y3\xbe\xd1\x84\xf9=\x10\x85\xaa=;d\x15\xbc\xe0Su=\x05D7>\x91\xa2\xde=\xf6z\xba\xbeC\x82\xa2\xbd\x99b\x17=\x0egW=J<\x9a>\x16V;\xbb\xe4.j>~\xca\x10\xbc\xcb\xbb\x04\xbe\x86 \x05=F^]\xbe\xa7\xf9\x01\xbeC\x01\x8f=5\x82X>\xa4\xa6\x00=\xfc\x10\x84<\xc0\x95\x1a\xbe\x88\xeb\x17>y\xab^=\x1f\xf8\xef\xbcE\xb2\x14\xbd*\xfe\x93=\xed\xd3T>\x10\xbd\xe8;nt\xb4\xbd9\x8d\x01\xbeA\xc6A>\xee\xe7\xec=\x7f#\x17\xbeB\x08\xd1\xbd\xf6\xfe\xdb=tp%\xbc\x94t\x94b.'

In [5]:
with open('./profiles_vectors.bin', 'bw') as file:
    pickle.dump(profiles_vectors, file)

In [6]:
with open('./profiles_vectors.bin', 'br') as file:
    profiles_vectors = pickle.load(file)

In [33]:
type(profiles_vectors[0]), profiles_vectors[0]

(bytes,
 b'\x80\x04\x95P\x01\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K2\x85\x94h\x03\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C\xc8\xd3I1\xbc\x95?c\xbe\xdd\x1c"\xbe\x055\xbf\xbdM\xba.\xbc\x1d\x92\x13\xbe\xcb\x88\xf2=\x97\x1aP>(\xbc\x8f\xbe\xaeM\xc8\xbdkF\x0e>\xd0y3\xbe\xd1\x84\xf9=\x10\x85\xaa=;d\x15\xbc\xe0Su=\x05D7>\x91\xa2\xde=\xf6z\xba\xbeC\x82\xa2\xbd\x99b\x17=\x0egW=J<\x9a>\x16V;\xbb\xe4.j>~\xca\x10\xbc\xcb\xbb\x04\xbe\x86 \x05=F^]\xbe\xa7\xf9\x01\xbeC\x01\x8f=5\x82X>\xa4\xa6\x00=\xfc\x10\x84<\xc0\x95\x1a\xbe\x88\xeb\x17>y\xab^=\x1f\xf8\xef\xbcE\xb2\x14\xbd*\xfe\x93=\xed\xd3T>\x10\xbd\xe8;nt\xb4\xbd9\x8d\x01\xbeA\xc6A>\xee\xe7\xec=\x7f#\x17\xbeB\x08\xd1\xbd\xf6\xfe\xdb=tp%\xbc\x94t\x94b.')

In [35]:
import numpy as np
pickle.loads(profiles_vectors[0])

array([-0.01082082, -0.22192223, -0.15831323, -0.09336285, -0.01066453,
       -0.14411207,  0.11842497,  0.20322643, -0.2807324 , -0.09780441,
        0.1389405 , -0.17526937,  0.12183536,  0.08326161, -0.00911814,
        0.05989444,  0.17897041,  0.10870851, -0.36421937, -0.07935002,
        0.03695926,  0.05258851,  0.30124122, -0.00285852,  0.2286945 ,
       -0.00883734, -0.12962262,  0.03250172, -0.21617994, -0.12692891,
        0.06982663,  0.2114342 ,  0.03140892,  0.01612138, -0.15096188,
        0.14835942,  0.05436275, -0.02929312, -0.03630282,  0.07226212,
        0.20783968,  0.00710262, -0.0881127 , -0.12651528,  0.18923284,
        0.11567675, -0.14759634, -0.10206653,  0.10741989, -0.01009761],
      dtype=float32)

In [39]:
model.dv.most_similar(pickle.loads(profiles_vectors[0]), topn=2000)[1:]

[('1', 0.9969642758369446),
 ('7', 0.9967203736305237),
 ('56', 0.9966126084327698),
 ('15', 0.9965221881866455),
 ('33', 0.9962326288223267),
 ('55', 0.9961355924606323),
 ('41', 0.9958972930908203),
 ('31', 0.9958867430686951),
 ('2', 0.9957226514816284),
 ('10', 0.9956842660903931),
 ('29', 0.9956255555152893),
 ('58', 0.9955169558525085),
 ('35', 0.9952890276908875),
 ('20', 0.9952616691589355),
 ('16', 0.9952535629272461),
 ('32', 0.9952259063720703),
 ('24', 0.995124340057373),
 ('38', 0.995004415512085),
 ('40', 0.9949389100074768),
 ('51', 0.9949057698249817),
 ('57', 0.994737982749939),
 ('27', 0.994559645652771),
 ('42', 0.9945456385612488),
 ('45', 0.9944214224815369),
 ('49', 0.9942367076873779),
 ('53', 0.9941332340240479),
 ('4', 0.9940277934074402),
 ('36', 0.9940208196640015),
 ('47', 0.9939458966255188),
 ('25', 0.9939031600952148),
 ('13', 0.9938251972198486),
 ('5', 0.9937857985496521),
 ('8', 0.9936375021934509),
 ('19', 0.9936362504959106),
 ('14', 0.99345541000366